In [44]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np

## Filtros

In [45]:
def filterImage (img, k):
    result = cv.GaussianBlur(img, (3,3), 0)
    return result

## Border detection

In [46]:
def borderDetectionImage (img, k):
    sobel = cv.Sobel(img, cv.CV_8UC1, dx=0, dy=1, ksize=k)
    return sobel

## Treshold

In [47]:
def treshold(img, treshLevel):
    result = cv.adaptiveThreshold(img, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, treshLevel, 3)
    return result

## Main

In [48]:
def processFrame (frame, gaussianFilterSize, sobelKSize, treshLevel, imageMode):
    ## Converte frame para tons de cinza
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray_3 = cv.cvtColor(gray, cv.COLOR_GRAY2BGR)  

    ## Filtros
    filtered = filterImage(gray, gaussianFilterSize)

    ## Passa pela detecção de bordas
    borderDetection = borderDetectionImage(filtered, sobelKSize)

    ## Filtra com um treshold
    result = treshold(borderDetection, treshLevel)

    ## Passa pela detecção de curvas
    
    ## Exibe na saida saida
    if(imageMode):
        gray_3 = cv.cvtColor(result, cv.COLOR_GRAY2BGR)    
        plt.imshow(gray_3)
        plt.title('Saida')
        plt.show()
    else:
        gray_3 = cv.cvtColor(result, cv.COLOR_GRAY2BGR)    
        imgOutput = np.concatenate((frame, gray_3), axis=1)
        cv.imshow('VisaoComputacional', imgOutput)

## Parâmetros

Nessa primeira parte definimos alguns parâmetros para a execução do programa

In [49]:
VIDEO = cv.VideoCapture("./assets/birds_low.mp4")
VIDEO_SAMPLE = cv.imread("./assets/birds_2.png")
IMG_MODE = False

# Valores default
gaussianFilterSize = 3
sobelKSize = 3
treshLevel = 11

### Calibração
Com base em uma imagem exemplo do vídeo podemos testar a melhor calibração e alterar

In [50]:
interact(processFrame, frame=fixed(VIDEO_SAMPLE), \
         gaussianFilterSize=widgets.IntSlider(min=1, max=30, step=2, value=3), \
         sobelKSize=widgets.IntSlider(min=1, max=31, step=2, value=3), \
         treshLevel=widgets.IntSlider(min=1, max=30, step=2, value=11), \
         imageMode=fixed(True))

interactive(children=(IntSlider(value=3, description='gaussianFilterSize', max=30, min=1, step=2), IntSlider(v…

<function __main__.processFrame(frame, gaussianFilterSize, sobelKSize, treshLevel, imageMode)>

## Fluxo principal

Entra no loop principal, chama a função para processar a imagem

In [51]:
if(IMG_MODE):
    image = VIDEO_SAMPLE
    processFrame(image, gaussianFilterSize, sobelKSize, treshLevel)
else:
    # Cria tela
    cv.namedWindow('VisaoComputacional', cv.WINDOW_NORMAL)
    cv.resizeWindow('VisaoComputacional', 800,600)

    while (VIDEO.isOpened()):
        ret, frame = VIDEO.read()
        if not ret:
            print("Saindo...")
            break

        ## Chama função para processar frame
        processFrame(frame, gaussianFilterSize, sobelKSize, treshLevel, False)
        if cv.waitKey(1) == ord('q'):
            break

    VIDEO.release()
    cv.destroyAllWindows()